-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Using Clone with Delta Lake

Delta Lake provides native support for copying existing tables with `CLONE`. This notebook will explore both deep and shallow clones. The docs for this feature are [here](https://docs.databricks.com/delta/delta-utility.html#clone-a-delta-table); full syntax docs are available [here](https://docs.databricks.com/spark/latest/spark-sql/language-manual/delta-clone.html).

## Learning Objectives
By the end of this lesson, you should be able to:
* Use deep clones to create full incremental backups of tables
* Use shallow clones to create development datasets
* Describe expected behavior after performing common database operations on source and clone tables

## Configure the environment
The following cell will create a database and source table that we'll use in this lesson, alongside some variables we'll use to control file locations.

In [0]:
%run ../Includes/clone-setup $mode="reset"

## Look at the production table details
The production table we'll be using as our source is named `sensors_prod`.

Use the following cell to explore the table history. Note that 4 total transactions have been run to create and load data into this table.

In [0]:
%sql
DESCRIBE HISTORY sensors_prod

Explore the table description to discover the schema and additional details. Note that comments have been added to describe each data field.

In [0]:
%sql
DESCRIBE FORMATTED sensors_prod

The helper function `check_files` was defined to accept a table name and return the count of underlying data files (as well as list the content of the table directory).

Recall that all Delta tables comprise:
1. Data files stored in parquet format
1. Transaction logs stored in the `_delta_log` directory

The table name we're interacting with in the metastore is just a pointer to these underlying assets.

In [0]:
check_files("sensors_prod")

## Create a backup of your dataset with deep clone

Deep clone will copy all data and metadata files from your source table to a specified location, registering it with the declared table name.

In [0]:
%sql
CREATE OR REPLACE TABLE sensors_backup 
DEEP CLONE sensors_prod
LOCATION '${c.userhome}/backup/sensors'

You'll recall that our `sensors_prod` table had 4 versions associated with it. The clone operation created version 0 of the cloned table. 

The `operationsParameters` field indicates the `sourceVersion` that was cloned.

The `operationMetrics` field will provide information about the files copied during this transaction.

In [0]:
%sql
DESCRIBE HISTORY sensors_backup

Metadata like comments will also be cloned.

In [0]:
%sql
DESCRIBE FORMATTED sensors_backup

## Incremental Cloning

If you examine the files in your backup table, you'll see that you have the same number of files as your source table. Upon closer examination, you'll note that file names and sizes have also been preserved by the clone. 

This allows Delta Lake to incrementally apply changes to the backup table.

In [0]:
check_files("sensors_backup")

To see incremental clone in action, begin by commiting a transaction to the `sensor_prod` table. Here, we'll delete all those records where `sensor_type` is `C`.

Remember that Delta Lake manages changes at the file level, so any file containing a matching record will be rewritten.

In [0]:
%sql
DELETE FROM sensors_prod WHERE sensor_type = 'C'

When we re-execute our deep clone command, we only copy those files that were written during our most recent transaction.

In [0]:
%sql
CREATE OR REPLACE TABLE sensors_backup 
DEEP CLONE sensors_prod
LOCATION '${c.userhome}/backup/sensors'

We can review our history to confirm this.

In [0]:
%sql
DESCRIBE HISTORY sensors_backup

## Creating Development Datasets with Shallow Clone

Whereas deep clone copies both data and metadata, shallow clone just copies the metadata and creates a pointer to the existing data files.

Note that the cloned table will have read-only permissions on the source data files. This makes it easy to create development datasets using a production dataset without fear of table corruption.

Here, we'll also specify using version 2 of our source production table.

In [0]:
%sql
CREATE OR REPLACE TABLE sensors_dev
SHALLOW CLONE sensors_prod@v2
LOCATION '${c.userhome}/dev/sensors'

When we look at the target directory, we'll note that no data files exist. The metadata for this table just points to those data files in the source table's data directory.

In [0]:
check_files("sensors_dev")

## Apply Changes to Dev Data
But what happens if you want to test modifications to your dev table?

The code below inserts only those records from version 3 of our production table that don't have the value "C" as a `sensor_type`.

In [0]:
%sql
MERGE INTO sensors_dev dev
USING (SELECT * FROM sensors_prod@v3 WHERE sensor_type != "C") prod
ON dev.device_id = prod.device_id AND dev.time = prod.time
WHEN NOT MATCHED THEN INSERT *

The operation is successful and new rows are inserted. If we check the contents of our table location, we'll see that data files now exists.

In [0]:
check_files("sensors_dev")

Any changes made to a shallow cloned table will write new data files to the specified target directory, meaning that you can safely test writes, updates, and deletes without risking corruption of your original table. The Delta logs will automatically reference the correct files (from the source table and this clone directory) to materialize the current view of your dev table.

## File Retention and Cloned Tables

It's important to understand how cloned tables behave with file retention actions.

Run the cell below to `VACUUM` your source production table (removing all files not referenced in the most recent version).

In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)
spark.sql("VACUUM sensors_prod RETAIN 0 HOURS")
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", True)

We see that there are now fewer total data files associated with this table.

In [0]:
check_files("sensors_prod")

You'll recall that our `sensors_dev` table was initialized against version 2 of our production table. As such, it still has reference to data files associated with that table version.

Because these data files have been removed by our vacuum operation, we should expect the following query against our shallow cloned table to fail.

In [0]:
%sql
SELECT * FROM sensors_dev

Because deep clone created a full copy of our files and associated metadata, we still have access to our `sensors_backup` table. Here, we'll query the original version of this backup (which corresponds to version 3 of our source table).

In [0]:
%sql
SELECT * FROM sensors_backup@v0

One of the useful features of deep cloning is the ability to set different table properties for file and log retention. This allows production tables to have optimized performance while maintaining files for auditing and regulatory compliance. 

The cell below sets the log and deleted file retention periods to 10 years.

In [0]:
%sql
ALTER TABLE sensors_backup
SET TBLPROPERTIES (
  delta.logRetentionDuration = '3650 days',
  delta.deletedFileRetentionDuration = '3650 days'
)

## Wrapping Up

In this notebook, we explored the basic syntax and behavior of deep and shallow clones. We saw how changes to source and clone tables impacted tables, including the ability to incrementally clone changes to keep a backup table in-sync with its source. We saw that shallow clone could be used for creating temporary tables for development based on production data, but noted that removal of source data files will lead to errors when trying to query this shallow clone.

Run the following cell to delete the tables and files associated with this demo.

In [0]:
%run ../Includes/clone-setup $mode="cleanup"

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>